<a href="https://colab.research.google.com/github/guhgah88/Projetos_Analises_de_dados/blob/main/Back_casa_PyCaret_football_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install lightgbm

### Importando as Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pycaret.classification import *

import warnings
warnings.filterwarnings('ignore')

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def plot_profit_acu(dataframe, title_text):
    dataframe['Profit_acu'] = dataframe.Profit.cumsum()
    n_apostas = dataframe.shape[0]
    profit = round(dataframe.Profit_acu.tail(1).item(), 2)
    ROI = round((dataframe.Profit_acu.tail(1) / n_apostas * 100).item(), 2)
    drawdown = dataframe['Profit_acu'] - dataframe['Profit_acu'].cummax()
    drawdown_maximo = round(drawdown.min(), 2)
    winrate_medio = round((dataframe['Profit'] > 0).mean() * 100, 2)
    desvio_padrao = round(dataframe['Profit'].std(), 2)
    dataframe.Profit_acu.plot(title=title_text, xlabel='Entradas', ylabel='Stakes')
    print("Método:",title_text)
    print("Profit:", profit, "stakes em", n_apostas, "jogos")
    print("ROI:", ROI, "%")
    print("Drawdown Maximo Acumulado:", drawdown_maximo)
    print("Winrate Medio:", winrate_medio, "%")
    print("Desvio Padrao:", desvio_padrao)
    print("")

def avg_total(df, team, date, var_for_column, var_against_column):
    prior_matches = df[(df['Home'] == team) | (df['Away'] == team)]
    prior_matches = prior_matches[prior_matches['Date'] < date].tail(5)

    var_for = 0
    var_against = 0

    for _, row in prior_matches.iterrows():
        if row['Home'] == team:
            var_for += row[var_for_column]
            var_against += row[var_against_column]
        else:
            var_for += row[var_against_column]
            var_against += row[var_for_column]

    num_games = len(prior_matches)
    avg_var_for = var_for / num_games if num_games > 0 else 0
    avg_var_against = var_against / num_games if num_games > 0 else 0

    return avg_var_for, avg_var_against

### Importando a Base de Dados

In [ ]:
base = pd.read_excel("https://github.com/guhgah88/Projetos_Analises_de_dados/raw/main/Planilhas/Base_de_dados_football_data.xlsx")
base["Data"] = pd.to_datetime(base["Data"])
base = base.sort_values('Data')
base = base[['Data','Home','Away','HG','AG','Result','H_odd','D_odd','A_odd','pro_H','pro_D','pro_A','CV_odd','CV_pro','Med_pts_H','Med_pts_A' ,
'CV_pts_H' ,'CV_pts_A',	'Média_GM_H','Média_GM_A','CV_GM_H','CV_GM_A','Média_GS_H','Média_GS_A','CV_GS_H','CV_GS_A','Média_SG_H',
'Média_SG_A','CV_SG_H','CV_SG_A','Média_CG_H','Média_CG_A','CV_CG_H','CV_CG_A']]

#[['Data','Home','Away','HG','AG','Result','H_odd','D_odd','A_odd','pro_H','pro_D','pro_A','CV_odd','CV_pro','Med_pts_H','Med_pts_A' ,
#'CV_pts_H' ,'CV_pts_A',	'Média_GM_H','Média_GM_A','CV_GM_H','CV_GM_A','Média_GS_H','Média_GS_A','CV_GS_H','CV_GS_A','Média_SG_H',
#'Média_SG_A','CV_SG_H','CV_SG_A','Média_CG_H','Média_CG_A','CV_CG_H','CV_CG_A']]



base.columns = ['Data','Home','Away','HG','AG','Result','H_odd','D_odd','A_odd','pro_H','pro_D','pro_A','CV_odd','CV_pro','Med_pts_H','Med_pts_A' ,
'CV_pts_H' ,'CV_pts_A',	'Média_GM_H','Média_GM_A','CV_GM_H','CV_GM_A','Média_GS_H','Média_GS_A','CV_GS_H','CV_GS_A','Média_SG_H',
'Média_SG_A','CV_SG_H','CV_SG_A','Média_CG_H','Média_CG_A','CV_CG_H','CV_CG_A']
#base = base[base['League'].isin(ligas) == True]
base = drop_reset_index(base)

### Criando Variáveis

In [ ]:
# Crie uma função para calcular os pontos com base no resultado
def calc_pts(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

# Aplique a função para criar as colunas de pontos
base['Pts_H'] = base['Result'].apply(calc_pts)
base['Pts_A'] = base['Result'].apply(lambda x: calc_pts('A' if x == 'H' else ('H' if x == 'A' else 'D')))

# Crie uma função para calcular a porcentagem de aproveitamento dos pontos
def calc_aproveitamento(df, team_col, pts_col, n_games):
    df['Aproveitamento_'+team_col] = 0
    teams = df[team_col].unique()
    for team in teams:
        team_games = df[df[team_col] == team]
        team_games['Aproveitamento_'+team_col] = team_games[pts_col].rolling(n_games).sum() / (3 * n_games)
        df.loc[df[team_col] == team, 'Aproveitamento_'+team_col] = team_games['Aproveitamento_'+team_col]
    return df

# Aplique a função ao seu DataFrame
base = calc_aproveitamento(base, 'Home', 'Pts_H', 5)
base = calc_aproveitamento(base, 'Away', 'Pts_A', 5)




### Criação do Modelo no Pycaret

In [ ]:
# Back Home
base.loc[(base['HG'] >  base['AG']), 'Back_Home'] = 1
base.loc[(base['HG'] <= base['AG']), 'Back_Home'] = 0

In [ ]:
treino = base[base['Data'].dt.year <= 2022]
teste = base[base['Data'].dt.year >= 2023]

In [ ]:
base_numeric = base.select_dtypes(include=[np.number])
corr = base_numeric.corr()

## Criando correlação com mapa de calor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcula a correlação
corr = base_numeric.corr()

# Seleciona apenas a coluna 'Back_Home'
corr_back_home = corr['Back_Home'].sort_values(ascending=False)

# Transforma a Series em DataFrame para o heatmap
corr_back_home = pd.DataFrame(corr_back_home)

# Cria um mapa de calor
plt.figure(figsize=(12,10))
sns.heatmap(corr_back_home, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

In [ ]:
features = ['Aproveitamento_Home','pro_H','CV_pro','A_odd','CV_odd']

label = 'Back_Home'

In [ ]:
# Configurando o ambiente PyCaret
Back_Home = setup(data = treino[features], target = treino[label], verbose = False)

In [ ]:
# Verificando os Modelos existentes no PyCaret
df_models = models()
modelos = df_models.index.tolist()
modelos

In [ ]:
for i in modelos:
    try:
        Treino = create_model(i, verbose=False)
        df1 = predict_model(Treino, teste, verbose=False)

        stake = 1
        green = stake * (df1.H_odd - 1)
        red = -stake

        df1.loc[(df1['prediction_label'] == 1) & (df1['Back_Home'] == 1), 'Profit'] = green
        df1.loc[(df1['prediction_label'] == 1) & (df1['Back_Home'] == 0), 'Profit'] = red
        df1.loc[(df1['prediction_label'] == 0) & (df1['Back_Home'] == 1), 'Profit'] = 0
        df1.loc[(df1['prediction_label'] == 0) & (df1['Back_Home'] == 0), 'Profit'] = 0

        filtro = (df1.prediction_label == 1) & (df1.A_odd >= 4) & (df1.Média_CG_A >=2)
        df0 = df1[filtro]
        df0 = drop_reset_index(df0)
        plt.figure()
        plot_profit_acu(df0,f'Back Home - {i}')
        plt.show()
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


In [ ]:
Treino = create_model('svm', verbose=False)
df1 = predict_model(Treino, teste, verbose=False)

In [ ]:
base_H = base[['Home', 'Aproveitamento_Home']]
base_A = base[['Away', 'Aproveitamento_Away']]

In [ ]:
jogos = pd.read_excel('https://www.football-data.co.uk/fixtures.xlsx')
jogos = jogos[['Date','HomeTeam','AwayTeam','B365H','B365D','B365A']]
jogos.columns = ['Data','Home','Away','H_odd','D_odd','A_odd']

jogos['pro_H'] = 1 / jogos.H_odd
jogos['pro_D'] = 1 / jogos.D_odd
jogos['pro_A'] = 1 / jogos.A_odd

#CV das Odds
desv_pad = jogos[['H_odd' , 'D_odd' , 'A_odd']].std(axis=1)
med_odd = jogos[['H_odd' , 'D_odd' , 'A_odd']].mean(axis=1)
cv_odd = desv_pad / med_odd
jogos['CV_odd'] = cv_odd

#CV das Probabilidades
desv_pad = jogos[['pro_H' , 'pro_D' , 'pro_A']].std(axis=1)
med_pro = jogos[['pro_H' , 'pro_D' , 'pro_A']].mean(axis=1)
cv_pro = desv_pad / med_pro
jogos['CV_pro'] = cv_pro



jogos

In [ ]:
ultima_base_H = base_H.groupby('Home').last().reset_index()
ultima_base_A = base_A.groupby('Away').last().reset_index()

df = pd.merge(jogos, ultima_base_H, how='left', left_on='Home', right_on='Home')
df = pd.merge(df, ultima_base_A, how='left', left_on='Away', right_on='Away')

df = drop_reset_index(df)
#df


In [ ]:
Treino = create_model('svm', verbose=False)
df1 = predict_model(Treino, df, verbose=False)
filtro = (df1.prediction_label == 1)
entradas = df1[filtro]
entradas= drop_reset_index(entradas)
entradas